# Test connectivity to a few different TD systems:

### Teradata Transcent (via TD-VPN)

In [ ]:
%rmconnect name=Transcend

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprd.td.teradata.com, props="LOGMECH=LDAP"

In [1]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


In [2]:
select * from dbc.dbcinfo;

InfoKey,InfoData
RELEASE,16.20.32.11
VERSION,16.20.32.11
LANGUAGE SUPPORT MODE,Standard


--

## CPU Work

In [20]:
drop table CSM_Alignment

Success: 0 rows affected

In [23]:
Create volatile table siteid_x_account as
(
    select
     Acct_Name  as AccountName
    ,a.Site_ID  as SiteID
    from adlste_coa.coat_mat_sales_role_qlid a
    join (
        Select distinct Site_ID from adlste_coa.coat_dim_cis_siteid
        where System_use  not in('test','development','QA')  /* prod only */
        qualify LogDate = max(logDate) over(partition by Site_ID)) b
    on a.Site_ID = b.Site_ID
    where assoc_qlid in(select QLID from adlste_coa.coat_dim_csm_territory)
    Group by 1,2
) with data no primary index on commit preserve rows;

Success: 0 rows affected

In [24]:
/*{{save:siteid_x_account.csv}}*/
select  * from siteid_x_account  order by 1,2;

AccountName,SiteID
American Airlines,AAMOSDR1
American Airlines,AAPREPROD1
American Airlines,AAPROD4
American Airlines,AAPROD5
"Apple, Inc.",APPLE41MEGADEW
"Apple, Inc.",APPLE46REDWOOD
"Apple, Inc.",APPLE48EDDIES
"Apple, Inc.",APPLE50BLANEY
"Apple, Inc.",APPLE57CRESCENT
"Apple, Inc.",APPLE62SEDONA


In [34]:

locking row for access
Select
  case when Account_Name is null then '== TOTALS ==' else Account_Name end as Account_Name
 ,case when Site_ID is null then '== SUBTOTALS ==' else Site_ID end as Site_ID
/* cpu slope growth */
  ,sum((yr2020q1_Time - yr2020q1_Time_avg) * (yr2020q1_CPU - yr2020q1_CPU_avg))
      / sum((yr2020q1_Time - yr2020q1_Time_avg) * (yr2020q1_Time - yr2020q1_Time_avg)) as yr2020q1_Slope
  ,sum((yr2019_Time - yr2019_Time_avg) * (yr2019_CPU - yr2019_CPU_avg))
      / sum((yr2019_Time - yr2019_Time_avg) * (yr2019_Time - yr2019_Time_avg)) as yr2019_Slope
  ,cast( (yr2020q1_Slope  / nullifzero(yr2019_Slope))-1 as decimal(9,4)) as  Slope_Growth
/* raw cpu growth, avg() to account different #of days */
  ,avg(yr2020q1_CPU) as  yr2020q1_CPUamt
  ,avg(yr2019_CPU) as yr2019_CPUamt
  ,cast((yr2020q1_CPUamt / nullifzero(yr2019_CPUamt))-1 as decimal(9,4)) as  CPU_Growth
from
(
    Select
     Account_Name, Site_ID, LogDate
    ,yr2020q1_CPU
    ,avg(yr2020q1_cpu) over(partition by Site_ID) as  yr2020q1_CPU_avg
    ,yr2020q1_Time
    ,avg(yr2020q1_time) over(partition by Site_ID) as  yr2020q1_Time_avg
    ,yr2019_CPU
    ,avg(yr2019_cpu) over(partition by Site_ID) as  yr2019_CPU_avg
    ,yr2019_Time
    ,avg(yr2019_time) over(partition by Site_ID) as  yr2019_Time_avg
    from
    (
      select
       AccountName as Account_Name
      ,Site_ID
      ,TheDate as LogDate
      ,Eff_Used_CPU_GHz as CPU
      ,case when TheDate between '2020-01-01' and '2020-03-30' then date_order else  null end as yr2020q1_Time
      ,case when TheDate between '2020-01-01' and '2020-03-30' then CPU        else  null end as yr2020q1_CPU
      ,case when TheDate between '2019-01-01' and '2019-12-31' then date_order else  null end as yr2019_Time
      ,case when TheDate between '2019-01-01' and '2019-12-31' then CPU        else  null end as yr2019_CPU
      ,ROW_NUMBER() OVER (partition by Site_ID order by TheDate Desc ) AS date_order
      from adlste_coa.coat_dim_cis_cpu_gtm_system_usage_hist as a
      join siteid_x_account   on a.Site_ID = SiteID
      where LogDate between '2019-01-01' and '2020-12-31'
    ) as a
) as b
Group by rollup(Account_Name, Site_ID) 
order by  1,2,3;


Account_Name,Site_ID,yr2020q1_Slope,yr2019_Slope,Slope_Growth,yr2020q1_CPUamt,yr2019_CPUamt,CPU_Growth
== TOTALS ==,== SUBTOTALS ==,0.46907973303479594,-2.5659061588021816,-1.1828,1990.1980271103287,1918.6069886013638,.0373
American Airlines,== SUBTOTALS ==,-0.2623449763923759,0.004820201728705628,-55.4261,98.2992782079278,7.980837849686201,11.3169
American Airlines,AAMOSDR1,0.0011164004154448554,2.1712974227047462e-05,50.4163,0.946481414141402,1.0196224781801977,-.0717
American Airlines,AAPREPROD1,-0.016412392424243055,0.009618690483184207,-2.7063,13.255562568181853,14.942053221192204,-.1129
American Airlines,AAPROD4,,,,,,
American Airlines,AAPROD5,-1.137977538521431,,,316.7148253579709,,
"Apple, Inc.",== SUBTOTALS ==,26.389379083487903,-3.3465186064989005,-8.8856,10121.875746507867,11759.938476246343,-.1393
"Apple, Inc.",APPLE41MEGADEW,-0.28435763880792586,108.45803760389636,-1.0026,9967.59865516181,14068.673336845039,-.2915
"Apple, Inc.",APPLE46REDWOOD,-14.639048742912799,13.121716356506768,-2.1156,14757.179895202305,14953.385585700942,-.0131
"Apple, Inc.",APPLE48EDDIES,30.3943378983379,-13.419614188019908,-3.2649,7870.344213333348,6398.301855139834,.2301


In [21]:
select *  from CSM_Alignment

QuicklookID,FullName,AccountName,SiteID
jo186015,Jeffery Ohlman,Flextronics International,FLEX01
rp210002,Rahul Patil,Sabre GLBL Inc.,SABREDEV2
pb186048,Patrice Berube,eBay Inc.,EBAY22
gg185047,Gerik Grant,American Airlines,AAMOSDR1
ts186006,Terry Stover,"Apple, Inc.",APPLE50BLANEY
rc250070,Ryan Coffman,Dell Inc.,DELLDEV3
js186127,Jim Summersgill,"PepsiCo, Inc.",TDCLOUDAZTDPHP2
yl186040,Yonatan Lerner,NortonLifeLock Inc.,SYMCTEST4
bb186023,Bruce Burton,AT&T Inc.,ATTTVMEDA02
dl185038,Deanne Larson,"T-Mobile USA, Inc.",TMONWKP1


QLID
pk151002
rp210002
ts186006
js186127
jo186015
gg185047
rc250070
rb186085
yl186040
mg186082


Success: 1 rows affected

In [9]:
select top 100
 a.* 
from adlste_coa.coa_dim_cis_cpu_gtm_customer_usage a
where cust_region_desc = 'Americas'
and sls_area_name = 'West'

gtm_acct_name,gtm_acct_nbr,entrp_acct_nbr,cust_nbr,acct_nbr,acct_name,cust_region_desc,sls_area_name,cvrg_model_cd,gtm_tier_name,strategic_sgmnt_cd,acct_industry_name,acct_subindustry_name,Max_Date,Total_CPU_Power,Used_CPU_GHz_Day,Used_CPU_GHz_Month,Used_CPU_GHz_Year,Growth_CPU_GHz_Month,SiteID_Count,Pct_Used_CPU_GHz,Growth_Pct_Month,Eff_XTotal_CPU_GHz,Eff_Used_CPU_GHz_Day,Eff_Used_CPU_GHz_Month,Eff_Used_CPU_GHz_Year,Eff_Growth_CPU_GHz_Month,Eff_Pct_Used_CPU_GHz,Eff_Growth_Pct_Month,Used_Daily_Avg_CPU,Used_Daily_CPU_GHz_Year,Growth_Daily_CPU_GHz_Month
"PayPal, Inc.",200596,101002,1909852,608513,"PayPal, Inc.",Americas,West,Enterprise,Strategic,Hyperscale Architect,Financial Services,,2020-04-22,43718.400,29728.293408000005,29063.987351999982,29284.770240000,327.013632000,1,0.664799886363636,.007480000,32788.8,39637.72454400009,38751.98313599997,39044.640729600,435.697574400,0.8863998484848479,0.013288,26310.70464,27714.143001600,838.693785600
Flextronics International,200358,102165,2461807,603137,Flextronics International,Americas,West,Enterprise,Named,Opportunistic Value Seeker,Manufacturing,Software,2020-04-27,830.880,276.79393319999974,310.7089698545445,320.289929280,-.274454400,3,0.3739516775642024,-.000330318,733.6800000000001,352.9050972,397.4775100363624,410.223257280,-.676473600,1.0696127272727238,-0.0009220281321557081,218.01095454285715,240.582786720,-1.306652160
American Airlines,200043,100074,167195,588345,American Airlines,Americas,West,Enterprise,Strategic,Analytic Visionary,Travel,Commercial Aviation,2020-04-27,3171.200,1080.1998879999978,1437.0926501818165,1328.196928000,-506.177196800,6,0.45316998302907935,-.159616926,3171.2,1080.1998879999978,1437.0926501818165,1328.196928000,-506.177196800,1.755863522727272,-0.1596169263370333,970.4957564444443,1037.726563200,-406.865888000
Activision Blizzard,200141,100011,5803053,617897,Blizzard Entertainment,Americas,West,Enterprise,Strategic,Opportunistic Value Seeker,Media & Entertainment,Gaming,2020-04-27,3494.400,2428.407072000007,2826.320356363644,2637.363456000,-21.217996800,1,0.8088142045454567,-.006072000,3494.4,2428.407072000007,2826.320356363644,2637.363456000,-21.217996800,0.8088142045454567,-0.006072,1962.070656,1936.872537600,-8.072064000
Sprint Nextel Corporation,200717,101202,28027,2344,Sprint Nextel Corporation,Americas,West,Enterprise,Strategic,Analytic Visionary,Communications,Telecom,2020-04-24,8608.000,3847.6708000000112,4192.901153454555,4360.578144000,-22.917171200,4,0.48709353548496226,-.002662311,8608,3847.670800000007,4192.901153454555,4288.499088000,-115.847916800,1.409347272727276,-0.0134581687732342,2826.65935584051,3113.429875200,60.256345600
AT&T Inc.,200081,100139,36464,2336,AT&T Inc.,Americas,West,Enterprise,Premier,Analytic Visionary,Communications,Telecom,2020-04-27,3564.240,488.1278338000009,517.5767058818184,623.963415520,-16.952656160,10,0.14521376391090904,-.004756317,2519.8559999999998,764.4158293630253,833.853737726847,1021.158548320,-23.510416160,1.3230309077267255,-0.009330063368700434,297.97773204891774,353.263040480,-1.530951840
"Williams-Sonoma, Inc.",200891,101442,29000,492678,"Williams-Sonoma, Inc.",Americas,West,Enterprise,Targeted,Conventional Conservative,Retail,Non Food,2020-04-24,1107.840,341.56598399999746,404.92498101817915,376.568830080,-14.858004480,2,0.36550854005829286,-.013411688,853.8240000000001,665.5322625882349,764.871822301599,720.207895680,-23.365985280,1.4684059915329666,-0.02736627838992579,314.3707057263158,295.791075840,-4.859036160
Albertson's LLC,200028,100043,27972,492513,Safeway Inc.,Americas,West,Enterprise,Targeted,Opportunistic Value Seeker,Retail,Food,2020-04-24,5831.040,5359.3468799999955,5206.829835054545,5094.904216320,69.567548160,2,0.8929504573891698,.011930556,5831.040000000001,5359.3468799999955,5206.829835054545,5094.904216320,69.567548160,1.4227199999999989,0.011930555811656238,5166.8867776,4008.965798400,72.494146560
NortonLifeLock Inc.,200764,101259,5401769,617473,NortonLifeLock Inc.,A

<br/><br/>
# Random bits of noise

In [ ]:
select tablename, tablekind, t.* from dbc.tables t where databasename = 'ADLSTE_COA' order  by 1

In [ ]:
select databasename, tablename, tablekind 
,substr(TableName, index(TableName, '_')) as ProcessName
from dbc.tables 
where databasename like 'adlste_westcomm%' and tablename like any('%consumption_cpu_forecast%')
    ----
order by databasename desc, TableKind, TableName

In [ ]:
Select SiteID, "Log Date" from adlste_westcomm.consumption_cpu_forecast_stg_boeing1

In [ ]:
insert into adlste_westcomm.consumption_cpu_forecast_stg
Select SiteID, "Log Date" from adlste_westcomm.consumption_cpu_forecast_stg_boeing1

In [ ]:
insert  into adlste_westcomm.consumption_storage_forecast_stg
Select * from adlste_westcomm.consumption_storage_forecast_stg_boeing3

In [ ]:
call adlste_westcomm.consumption_storage_forecast_sp('1.0')

In [ ]:
Select top  10 * from adlste_westcomm.consumption_storage_forecast_stg_boeing3

In [ ]:
Select top  10 * from adlste_westcomm.consumption_storage_forecast_stg

In [ ]:
Select * from adlste_coa_STG.stg_dat_dbql_core

In [ ]:
-- make test records:
delete from adlste_coa_STG.stg_dat_dbql_core;
 
insert into adlste_coa_STG.stg_dat_dbql_core
Select * from adlste_coa.coat_dat_dbql_core where SiteID = 'test' and LogDate = '2020-04-19';

update adlste_coa_STG.stg_dat_dbql_core set SiteID = 'test-'|| trim(USER);
 
Select 'STG' as TableType, count(*) from adlste_coa_stg.stg_dat_dbql_core ;
 

In [ ]:
-- execute test:
Call adlste_coa.sp_dat_DBQL_Core('test');

In [ ]:
-- both of these record counts SHOULD match
Select 'TGT' as TableType, count(*) from adlste_coa.coa_dat_dbql_core    where SiteID = 'test' and LogDate = '2020-04-19'
union all
Select 'STG' as TableType, count(*) from adlste_coa_stg.stg_dat_dbql_core where SiteID = 'test-'|| trim(USER) and LogDate = '2020-04-19';

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_DBQL_Core ( fileset_version VARCHAR(128) )
SQL SECURITY CREATOR
BEGIN

  delete from adlste_coa.coat_dat_DBQL_Core
  where   (SiteID, LogDate) in
   (Select SiteID, LogDate From adlste_coa_stg.stg_dat_DBQL_Core);

  Insert into adlste_coa.coat_dat_DBQL_Core
  Select * From adlste_coa_stg.stg_dat_DBQL_Core;
END;

In [ ]:
-- validate...
Select 'CPU' as Metric, SiteID, max("Log Date") as LogDate_Max, min("Log Date") as LogDate_Min, count(*) as RecCount
from adlste_westcomm.consumption_CPU_forecast_v2 where "Peak Start" is not null and SiteID like 'Boeing%'
group by SiteID
    ---
    union all
    ---
Select 'IO' as Metric, SiteID, max("Log Date") as LogDate_Max, min("Log Date") as LogDate_Min, count(*) as RecCount
from adlste_westcomm.consumption_IO_forecast_v2 where "Peak Start" is not null and SiteID like 'Boeing%'
group by SiteID
order by 1,2,3;